In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import Food101
import torch
import random
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models
import os
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from pathlib import Path
from torch.optim import lr_scheduler
from torchmetrics import Precision, Recall, F1Score, ConfusionMatrix
from torchinfo import summary


In [ ]:
root = 'data'  # Directory where the dataset will be stored
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),# Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize using ImageNet stats
])

food101_train = Food101(root=root, split='train', transform=transform, download=True)
food101_test = Food101(root=root, split='test', transform=transform, download=True)

In [ ]:
def Food101_dataloaders(
    train_dataset,
    test_dataset,
    batch_size:int,
    num_workers:int):

  class_names = train_dataset.classes
  class_to_idx=train_dataset.class_to_idx

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_dataset,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )


  test_dataloader = DataLoader(
      test_dataset,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names,class_to_idx

In [ ]:
def EfficientNet_V2(num_classes:int,
                    freeze_percentage:float=0.5,
                    tr_learning=False
                    ):
  weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
  #transform=weights.transforms
  EfficientNet_V2=models.efficientnet_v2_s(weights=weights)
  if tr_learning:
    for param in EfficientNet_V2.parameters():
      param.requires_grad=False

    EfficientNet_V2.classifier=nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=num_classes,bias=True)
  )
  else:
    params = list(EfficientNet_V2.parameters())

    # Calculate the number of parameters to freeze
    num_params_to_freeze = int(len(params) * freeze_percentage)

    # Randomly select parameters to freeze
    params_to_freeze = random.sample(params, num_params_to_freeze)

    # Freeze selected parameters
    for param in params_to_freeze:
       param.requires_grad = False

    EfficientNet_V2.classifier=nn.Sequential(
       nn.Dropout(p=0.2,inplace=True),
       nn.Linear(in_features=1280,out_features=num_classes,bias=True)
  )
  return EfficientNet_V2

In [ ]:
Num_Workers=os.cpu_count()

Food101_train_dataloader,Food101_test_dataloader,class_names,class_to_idx=Food101_dataloaders(train_dataset=food101_train,
                                                                              
                                                                              test_dataset=food101_test,
                                                                              batch_size=40,
                                                                              num_workers=Num_Workers
                                                                              )